In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding

2022-09-12 18:08:09.617690: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-12 18:08:09.617736: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
cols = ['OriginalTweet', 'Sentiment']

In [3]:
df_train = pd.read_csv('data/Corona_NLP_train.csv', encoding='latin1', usecols=cols)

In [4]:
df_train.head(3)

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive


In [5]:
df_train['Sentiment'].unique()

array(['Neutral', 'Positive', 'Extremely Negative', 'Negative',
       'Extremely Positive'], dtype=object)

In [6]:
#encode the target variable
df_train['Sentiment'] = df_train['Sentiment'].replace({'Extremely Negative':0, 'Negative':1, 'Neutral':2, 'Positive':3, 'Extremely Positive':4})
df_train.head(3)

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,2
1,advice Talk to your neighbours family to excha...,3
2,Coronavirus Australia: Woolworths to give elde...,3


In [7]:
vocab_size = 5000
encoded_docs = [one_hot(d, vocab_size) for d in df_train['OriginalTweet']]

In [8]:
encoded_docs[0]

[1152,
 1266,
 3565,
 2881,
 4408,
 4140,
 2286,
 4217,
 356,
 4408,
 4140,
 2286,
 3215,
 356,
 4408,
 4140,
 2286,
 74]

In [9]:
#input length is the max length of the sentence
max_length = df_train['OriginalTweet'].str.len().max()
embeded_vecotr_size = 5
model = Sequential()
model.add(Embedding(vocab_size, embeded_vecotr_size, input_length=max_length))
model.add(Flatten())
#add 3 hidden layers
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
#add output layer
#5 classes
model.add(Dense(5, activation='softmax'))  
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(model.summary())


2022-09-12 18:08:16.149803: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-12 18:08:16.149850: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-12 18:08:16.149884: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pops): /proc/driver/nvidia/version does not exist
2022-09-12 18:08:16.150254: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 355, 5)            25000     
                                                                 
 flatten (Flatten)           (None, 1775)              0         
                                                                 
 dense (Dense)               (None, 10)                17760     
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 5)                 55        
                                                                 
Total params: 43,035
Trainable params: 43,035
Non-traina

In [10]:
#pad the sentences to the max length
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[1152 1266 3565 ...    0    0    0]
 [1927  306 4321 ...    0    0    0]
 [1927 4605  138 ...    0    0    0]
 ...
 [4095 3988 3377 ...    0    0    0]
 [ 116 2880 4505 ...    0    0    0]
 [3901 4732 4337 ...    0    0    0]]


In [11]:
X = padded_docs
y = df_train['Sentiment']

In [12]:
#split the data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
#train the model
model.fit(X_train, y_train, epochs=10, verbose=1)

Epoch 1/10
1029/1029 [==============================] - 5s 4ms/step - loss: 1.5832 - accuracy: 0.2771
Epoch 2/10
1029/1029 [==============================] - 4s 4ms/step - loss: 1.5757 - accuracy: 0.2774
Epoch 3/10
1029/1029 [==============================] - 4s 4ms/step - loss: 1.5757 - accuracy: 0.2774
Epoch 4/10
1029/1029 [==============================] - 4s 4ms/step - loss: 1.5757 - accuracy: 0.2774
Epoch 5/10
1029/1029 [==============================] - 4s 4ms/step - loss: 1.5757 - accuracy: 0.2774
Epoch 6/10
1029/1029 [==============================] - 4s 4ms/step - loss: 1.5756 - accuracy: 0.2774
Epoch 7/10
 204/1029 [====>.........................] - ETA: 3s - loss: 1.5782 - accuracy: 0.2679

In [ ]:
#evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

258/258 [==============================] - 1s 2ms/step - loss: 1.5064 - accuracy: 0.3110
Accuracy: 31.098154
